# **Signa_Lab ITESO:** Postprocesador de datos de redes sociodigitales
## **Cuaderno: YouTube (listas de videos o canales)**

Cuaderno de código abierto diseñado para importar datos sobre listas de videos o canales desde la API de YouTube, como las arrojadas por la herramienta YouTube Data Tools, y preparar sus datos para análisis exploratorio, con herramientas para la visualización de datos y técnicas de análisis semántico.

**\***Los grupos de celdas marcadas con **asterisco requieren información** antes de seguir adelante.

## **0. Introducción**

Antes de comenzar, necesitas obtener al menos un archivo en formato **CSV o Excel** que registre **listas de videos o canales de YouTube** y sus metadatos.

Puedes leer la documentación de la YouTube Data API para saber más sobre los datos consultables, los operadores de búsqueda y cómo hacer implementaciones propias.

Para descargar fácilmente datos de videos o canales, puedes usar la herramienta YouTube Data Tools, desarrollada por Bernhard Rieder desde Digital Methods Initiative.

**Los siguientes módulos de [YouTube Data Tools](https://ytdt.digitalmethods.net/) son compatibles con este código:**


*   YouTube **Video List** ([**descargar datos**](https://ytdt.digitalmethods.net/mod_videos_list.php) / [documentación de API](https://developers.google.com/youtube/v3/docs/videos))

*   YouTube **Channel List** ([**descargar datos**](https://ytdt.digitalmethods.net/mod_channels_list.php) / [documentación de API](https://developers.google.com/youtube/v3/docs/channels))


Puedes consultar en este [video un tutorial](https://ytdt.digitalmethods.net/)
sobre cómo usar los distintos módulos de Youtube Data Tools, publicado por su creador, Bernhard Rieder.


---

## **1. Importar librerías y archivos de datos**

### Instalar e importar librerías:

**Instalar** librerías.

In [ ]:
# Instalar librerías de Python necesarias

!pip install pandas
!pip install nltk
!pip install difflib
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install numpy
!pip install plotly
!pip install time
!pip install tqdm
!pip install operator
!pip install wordcloud
!pip install networkx
# !pip install scikit-learn



**Importar librerías** necesarias:

In [ ]:
# Importar librerías de Python necesarias

import pandas as pd
import nltk
import re
import sys
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.stats import gaussian_kde
import numpy as np
import plotly.express as px
import time
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime
import math
import operator
from IPython.display import display, clear_output
import ipywidgets as widgets
import plotly.graph_objects as go
from wordcloud import WordCloud

from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramCollocationFinder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import networkx as nx
import plotly.graph_objects as go

### *Indicar rutas de archivos de datos a importar y nombre de proyecto:

**Importar y concatenar archivos de datos** (en CSV o Excel):

In [ ]:
# Definir función para cargar archivos a partir de la extensión en su ruta indicada
def load_file(path):
    if path.endswith('.csv'):
        return pd.read_csv(path)
    elif path.endswith('.xlsx'):
        return pd.read_excel(path)
    else:
        raise ValueError("Formato no compatible. Por favor carga solo archivos .csv or .xlsx.")

# Inicializar lista para alojar todas las rutas y una variable para el DataFrame final, accesible globalmente
file_paths = []
dfs = []
df = None  # DataFrame global

# Definir función para añadir un nuevo campo de texto (input) para añadir una ruta de archivo adicional
def add_file_input(b=None):
    path_input = widgets.Text(value='', placeholder='Escribe la ruta del archivo', description=f'Archivo {len(file_paths) + 1}:')
    file_paths.append(path_input)
    update_ui()

# Definir función para eliminar el último campo de texto (input) para ruta de archivo
def remove_file_input(b=None):
    if file_paths:
        file_paths.pop()
        update_ui()

# Definir función para procesar y cargar todos los archivos
def process_files(b):
    global dfs, df
    dfs = []  # Vaciar DataFrames

    for path_input in file_paths:
        path = path_input.value
        try:
            temp_df = load_file(path)
            temp_df['filename'] = path  # Add a column with the filename
            dfs.append(temp_df)
            print(f"Nombre de archivo: {path}")
            print(f"Filas/Columnas (shape): {temp_df.shape}")
        except ValueError as e:
            print(f"Error al cargar el archivo {path}: {e}")
            return

    if dfs:
        df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
        print("\n¡Se cargaron todos los archivos!")
        print(f"Filas/Columnas (shape) de DataFrame creado: {df.shape}")

# Campo de texto (input) para indicar nombre del proyecto (para integrarse en nombres de archivos a exportar)
project_name = widgets.Text(value='', placeholder='Escribe el nombre del proyecto (corto y sin espacios)', description='Nombre proyecto:')

# Botones para añadir y eliminar archivos
add_button = widgets.Button(description="Añadir archivo",  button_style='')
remove_button = widgets.Button(description="Eliminar archivo",  button_style='warning')
load_button = widgets.Button(description="Cargar archivos",  button_style='primary')

add_button.on_click(add_file_input)
remove_button.on_click(remove_file_input)
load_button.on_click(process_files)

# Definir función para actualizar UI
def update_ui():
    clear_output()
    display(project_name)
    for path_input in file_paths:
        display(path_input)
    display(widgets.HBox([add_button, remove_button]))
    display(load_button)

# Inicializar UI con un campo de texto (input) para ruta de archivo
add_file_input()


### Previsualizar datos importados:

**Previsualizar tabla** con todos los registros importados:

In [ ]:
# Previsualizar dataframe con CSVs importados
display(df)
print(f"Filas/Columnas (shape) en registros importados: {df.shape}")


**Exportar copia en CSV con registros importados (o concatenados):**

In [ ]:
# Exportar archivo CSV con tabla de registros importados (y concatenados, en el caso de múltiples archivos)
df.to_csv(f"{project_name.value}_registros-importados.csv")

---

## **2. Limpieza de registros importados**

### Generar identificadores únicos (IDs) por registro:

In [ ]:
# Definir función para asignar IDs únicos a cada fila en el data frame indicado como parámetro, comenzando desde '1000001'.
def assign_unique_ids(df):
    # Inicializar contador para IDs
    id_counter = 1000001

    # Crear copia de data frame original
    df_copy = df.copy()

    # Verificar si el data frame tiene una columna llamada 'videoId'
    if 'videoId' in df.columns:
        # Si la columna 'videoId' existe, no hacer nada
        pass
    else:
        # Si la columna 'videoId' no existe, duplicar la columna 'id' a una nueva columna llamada 'channelId'
        df_copy['channelId'] = df_copy['id'].copy()

    # Iterar a través de las filas del data frame
    for index, _ in enumerate(df_copy.index):
        # Dar formato a ID con ceros adicionales e incorporarlo al data frame
        formatted_id = str(id_counter).zfill(7)  # Se asegura de que sea un ID de 7 dígitos, agregando ceros cuando sea necesario
        df_copy.loc[index, 'id'] = formatted_id

        # Incrementar el contador del ID para la siguiente iteración
        id_counter += 1

    return df_copy

In [ ]:
# Ejecutar función para asignar IDs a cada registro y previsualizar tabla
if __name__ == "__main__":
    # Invocar la función con data frame de trabajo
    df_ids = assign_unique_ids(df)

# Sobreescribir data frame con nueva tabla con IDs generados
df = df_ids
df

### *Elegir columna y criterios para limpieza de texto sin aporte semántico:

In [ ]:
# Definir función para limpiar usuarios, hashtags y URLs
def limpiar_texto(texto, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_personalizado):
    # Eliminar usuarios si está activado
    if eliminar_usuarios:
        texto = re.sub(r"(?<!\w)@(\w+)(?!\w)", "", texto)

    # Eliminar hashtags si está activado
    if eliminar_hashtags:
        texto = re.sub(r"(?<!\w)#(\w+)(?!\w)", "", texto)

    # Eliminar URLs si está activado
    if eliminar_urls:
        texto = re.sub(r"(http|https|ftp)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", "", texto)
        texto = texto.lstrip(". ")

    # Aplicar regex personalizado si se proporciona
    if regex_personalizado:
        texto = re.sub(regex_personalizado, "", texto)

    return texto.strip()

# Definir función para agregar nueva columna con texto limpio (clean_text)
def agregarCleanTextADf(df, colText, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_personalizado):
    dfW = df.copy()
    dfW["clean_text"] = None

    for index, row in dfW.iterrows():
        text = str(row[colText])
        cleaned_text = limpiar_texto(text, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_personalizado)
        dfW.at[index, "clean_text"] = cleaned_text

    return dfW


# Crear widget de desplegable con columnas disponibles
available_columns = list(df.columns)
column_dropdown = widgets.Dropdown(options=available_columns, value=available_columns[0], description='Columna:')


# Definir widgets de IPyWidgets para la UI interactiva
eliminar_usuarios = widgets.Checkbox(value=True, description='Eliminar usuarios @')
eliminar_hashtags = widgets.Checkbox(value=True, description='Eliminar hashtags #')
eliminar_urls = widgets.Checkbox(value=True, description='Eliminar URLs')
regex_input = widgets.Text(value='', description='Otro (Regex):', placeholder='Escribe un regex opcional')


# Botón para ejecutar la limpieza
boton_limpiar = widgets.Button(description="Limpiar texto", button_style='warning')

# Caja de texto para mostrar el resultado
output_resultado = widgets.Output()

# Función que se ejecuta al hacer clic en el botón de limpiar texto
def ejecutar_limpieza(b):
    global dfCleanText  # Hacer que dfCleanText sea accesible globalmente
    with output_resultado:
        output_resultado.clear_output()  # Limpiar cualquier salida previa

        # Obtener el nombre de la columna seleccionada por el usuario
        text_column = column_dropdown.value

        # Verificar si la columna existe en el DataFrame
        if text_column not in df.columns:
            print(f"Error: La columna '{text_column}' no existe en el DataFrame.")
            return

        # Ejecutar la función de limpieza sobre el DataFrame seleccionado
        dfCleanText = agregarCleanTextADf(
            df, text_column,
            eliminar_usuarios.value,
            eliminar_hashtags.value,
            eliminar_urls.value,
            regex_input.value
        )

        # Mostrar el DataFrame con la nueva columna 'clean_text'
        print("Texto limpio aplicado. DataFrame actualizado globalmente como 'dfCleanText'.")
        display(dfCleanText)

# Conectar el botón con la función de limpieza
boton_limpiar.on_click(ejecutar_limpieza)

# Desplegar los widgets en pantalla
display(column_dropdown, eliminar_usuarios, eliminar_hashtags, eliminar_urls, regex_input, boton_limpiar, output_resultado)


### \*Elegir idioma de palabras vacías (*stop words*) a eliminar:

Elegir **idioma de** (desde diccionarios de NLTK), **eliminar *stop words*** y **agregar columna sem_text** con texto depurado:

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

# Definir función para eliminar stopwords y signos de puntuación
def delete_stopwords(texto, stopwords_list):
    # Tokenizar el texto
    tokens = nltk.word_tokenize(texto)

    # Eliminar signos de puntuación
    tokens = [token for token in tokens if token.isalnum()]

    # Eliminar stop words
    tokens = [token for token in tokens if token.lower() not in stopwords_list]

    # Convertir la lista de tokens a un string
    texto_limpio = " ".join(tokens)

    return texto_limpio.strip()

# Definir función para eliminar palabras vacías y agregar nueva columna con el resultado (sem_text)
def agregarSemTextADf(df, colText, stopwords_list):
    dfW = df.copy()
    dfW["sem_text"] = None

    for index, row in dfW.iterrows():
        text = row[colText]
        cleaned_text = delete_stopwords(text, stopwords_list)
        dfW.at[index, "sem_text"] = cleaned_text

    return dfW

# Lista de idiomas disponibles en NLTK
idiomas_stopwords = [
    'arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french',
    'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali',
    'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish',
    'swedish', 'tajik', 'turkish'
]

# Crear dropdown para seleccionar idioma
dropdown_idioma = widgets.Dropdown(
    options=idiomas_stopwords,
    value='spanish',  # Valor por defecto
    description='Idioma:'
)

# Botón para eliminar stopwords
boton_eliminar_stopwords = widgets.Button(description="Eliminar stopwords", button_style='warning')

# Output para mostrar el resultado
output_resultado_stopwords = widgets.Output()

# Función que se ejecuta al hacer clic en el botón de eliminar stopwords
def ejecutar_eliminacion_stopwords(b):
    global dfClean  # Hacer que dfClean sea accesible globalmente
    with output_resultado_stopwords:
        output_resultado_stopwords.clear_output()  # Limpiar cualquier salida previa

        # Obtener el idioma seleccionado
        idioma_seleccionado = dropdown_idioma.value

        # Descargar stopwords del idioma seleccionado
        stopwords_list = stopwords.words(idioma_seleccionado)

        # Verificar si la columna 'clean_text' existe en el DataFrame
        if 'clean_text' not in dfCleanText.columns:
            print("Error: La columna 'clean_text' no existe en el DataFrame.")
            return

        # Ejecutar la función para eliminar stopwords y agregar la nueva columna 'sem_text'
        dfClean = agregarSemTextADf(dfCleanText, 'clean_text', stopwords_list)

        # Mostrar el DataFrame con la nueva columna 'sem_text'
        print(f"Stopwords en {idioma_seleccionado} eliminadas. DataFrame actualizado globalmente como 'dfClean'.")
        display(dfClean)

# Conectar el botón con la función de eliminación de stopwords
boton_eliminar_stopwords.on_click(ejecutar_eliminacion_stopwords)

# Desplegar los widgets en pantalla
display(dropdown_idioma, boton_eliminar_stopwords, output_resultado_stopwords)

# Exportar archivo CSV con tabla completa de registros importados con IDs y texto sin palabras vacías (sem_text)
# dfClean.to_csv(f"{project_name.value}_registros-semtext.csv")


### Procesar duraciones, ratio de alcance y/o concatenación de URLs

In [ ]:
# Definir función para procesar listas de videos o canales

def listParse(df):
    # Comprobamos si el dataframe tiene una columna 'videoId'
    if 'videoId' in df.columns:
        # Si es una lista de videos, creamos las columnas adicionales
        # durationMin: duración en minutos con 2 decimales
        df['durationMin'] = df['durationSec'] / 60.0
        df['durationMin'] = df['durationMin'].round(2)

        # durationRuntime: duración en formato hh:mm:ss
        df['durationRuntime'] = df['durationSec'].apply(lambda x: '{:02d}:{:02d}:{:02d}'.format(int(x // 3600), int((x % 3600) // 60), int(x % 60)))

        # videoURL: URL completa del video
        df['videoURL'] = 'https://www.youtube.com/watch?v=' + df['videoId']

        # channelURL: URL completa del canal
        df['channelURL'] = 'https://www.youtube.com/channel/' + df['channelId']

        # Eliminamos registros duplicados según videoId
        df = df.drop_duplicates(subset='videoId', keep='first')
    else:
        # Si es una lista de canales, creamos las columnas adicionales
        # channelURL: URL completa del canal
        # df['channelURL'] = 'https://www.youtube.com/channel/' + df['id']

        # reachRatio: relación de alcance (viewCount / videoCount) * 100
        df['reachRatio'] = (df['videoCount'] / df['viewCount']) * 100
        df['reachRatio'] = df['reachRatio'].round(4)

        # Eliminamos registros duplicados según channelId
        df = df.drop_duplicates(subset='id', keep='first')

    return df

listParse(dfClean)

---

## **3. Depuración de registros desde diccionarios personalizados (con términos de filtrado o descarte)**







### *Definir uso de de diccionario de depuración:

A partir de diccionarios personalizados con términos de descarte o filtración, elige depurar o mantener, respectivamente, registros que los contengan.

El diccionario debe cargarse en formato CSV y contener, al menos, los siguientes campos:

| palabra | tipo | categoría | diccionario |
|---------|------|-----------|-------------|
|         |      |           |             |
|         |      |           |             |
|         |      |           |             |

Ejemplo:

| palabra | tipo   | categoría   | diccionario        |
|---------|--------|-------------|--------------------|
| idiota  | ofensa | humillación | ofensa-humillación |
| zorra   | ofensa | género      | ofensa-género      |
|         |        |             |                    |

Puedes [encontrar aquí una copia del diccionario](https://drive.google.com/file/d/1zK214W0pBRYn9lnY_MDJYhEEc6pI3L6F/view?usp=sharing) con la estructura requerida, en formato CSV, para descargar, llenar e incorporar a este cuaderno de código.

**Nota:** La versión actual de este cuaderno de código permite elegir entre las siguientes opciones sobre la carga de diccionarios:


- ***No cargar diccionario:*** Ignorar esta funcionalidad y no usar diccionarios.
- ***Diccionario de descarte (negativo):*** Se eliminarán todos los registros que mencionen alguno de esos términos.
- ***Diccionario de filtrado (positivo):*** Se mantendrán solo los registros que mencionen alguno de esos términos.

In [ ]:
# Create the dropdown widget for dictionary usage selection
cargar_diccionario = widgets.Dropdown(
    options=["No cargar diccionario", "Diccionario de descarte (negativo)", "Diccionario de filtrado (positivo)"],
    value="No cargar diccionario",
    description="Elegir uso:"
)

# Create the text input widget for file path
rutaDicc = widgets.Text(
    value="",
    placeholder="Escribir ruta a archivo CSV",
    description="Ruta:"
)

# Create the "Aceptar" button
accept_button = widgets.Button(description="Aceptar")

# Create an output widget to display results or errors
output = widgets.Output()

# Function to load the dictionary based on user input
def load_dictionary(b):
    with output:
        clear_output()
        if cargar_diccionario.value != "No cargar diccionario":
            try:
                global dfDiccionario
                dfDiccionario = pd.read_csv(rutaDicc.value)
                print(f"Archivo de diccionario '{rutaDicc.value}' cargado exitosamente.")
                display(dfDiccionario.head())  # Show the first few rows of the loaded dictionary
            except FileNotFoundError:
                print(f"El archivo de diccionario '{rutaDicc.value}' no fue encontrado. Por favor, verifique la ruta.")
        else:
            dfDiccionario = None
            print("No se cargó ningún diccionario.")

# Link the button to the load function
accept_button.on_click(load_dictionary)

# Display the widgets
display(cargar_diccionario, rutaDicc, accept_button, output)


In [ ]:
# Verificar uso de diccionario elegido
print(f"Uso de diccionario elegido: {cargar_diccionario.value}")

### Aplicar filtrado por diccionario y previsualizar resultados (opcional, solo si se cargó algún diccionario):

**Previsualizar diccionario de descarte importado:**

In [ ]:
if cargar_diccionario.value != "No cargar diccionario":
  # Previsualizar tabla de diccionario cargado (en caso de haber elegido utilizar diccionarios)
  try:
      display(dfDiccionario.head())  # Previsualizar primeros registros de diccionario
      print(f"\nFilas y columnas en diccionario cargado:")  # Verificar número de filas y columnas en diccionario
      print(f"Shape: {dfDiccionario.shape}")  # Verificar número de filas y columnas en diccionario
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")


**Filtrar registros a partir de diccionario cargado y modalidad de depuración (positiva o negativa):**

In [ ]:
def filtrar_registros(df_registros, df_terminos, colTexto, cargar_diccionario):
    # Definir los términos del diccionario
    terminos = df_terminos["palabra"].tolist()
    # Compilar expresiones regulares una sola vez
    expresiones_regex = [
        re.compile(r"(?<!\S)?(?:\s|[.,;:?!¡¿]){}(?:\s|[.,;:?!¡¿])?(?!\S)".format(re.escape(termino)), re.IGNORECASE)
        for termino in terminos
    ]

    # Copiar el dataframe de registros y agregar columnas auxiliares
    df_registros_filtrados = df_registros.copy()
    df_registros_filtrados["contiene_termino"] = False
    df_registros_filtrados["razon_eliminacion"] = ""

    # Filtrar registros
    for i in range(df_registros_filtrados.shape[0]):
        texto = str(df_registros_filtrados.loc[i, colTexto]).lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

        # Buscar coincidencias con expresiones regulares
        for expresion, razon in zip(expresiones_regex, df_terminos["categoría"]):
            coincidencias = expresion.findall(f" {texto} ")
            if coincidencias:
                df_registros_filtrados.loc[i, "contiene_termino"] = True
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Presencia de términos relacionados a {razon}"
                break
            else:
                df_registros_filtrados.loc[i, "contiene_termino"] = False
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Ausencia de términos relacionados a {razon}"

    # Revisar uso elegido del diccionario, para descarte (negativo) o filtrado (positivo)
    if cargar_diccionario == "Diccionario de descarte (negativo)":
        # Mantener registros que no contengan términos del diccionario (negativo)
        df_registros_filtrados_final = df_registros_filtrados[~df_registros_filtrados["contiene_termino"]]
        df_registros_eliminados = df_registros_filtrados[df_registros_filtrados["contiene_termino"]]
    elif cargar_diccionario == "Diccionario de filtrado (positivo)":
        # Mantener registros que sí contengan al menos un término del diccionario (positivo)
        df_registros_filtrados_final = df_registros_filtrados[df_registros_filtrados["contiene_termino"]]
        df_registros_eliminados = df_registros_filtrados[~df_registros_filtrados["contiene_termino"]]

    # Eliminar columnas auxiliares para tabla con registros depurados
    df_registros_filtrados_final = df_registros_filtrados_final.drop(columns=["contiene_termino", "razon_eliminacion"])

    df_registros_filtrados_final = df_registros_filtrados_final.reset_index(drop=True)
    df_registros_eliminados = df_registros_eliminados.reset_index(drop=True)

    return df_registros_filtrados_final, df_registros_eliminados


In [ ]:
# Definir el nombre de la columna que contiene el texto, si no está ya definido
# Asegúrate de que esta columna exista en el DataFrame dfClean
if 'text_column' not in globals():
    text_column = 'sem_text'  # Cambia 'sem_text' por el nombre correcto de la columna

# Ejecutar depuración de registros por diccionario cargado
if cargar_diccionario.value != "No cargar diccionario":
    try:
        # Asegúrate de que dfDiccionario esté definido y cargado antes de la función
        if 'dfDiccionario' in globals() and isinstance(dfDiccionario, pd.DataFrame):
            df_depurados_dicc, df_eliminados_dicc = filtrar_registros(dfClean, dfDiccionario, text_column, cargar_diccionario.value)
        else:
            print("El diccionario no está cargado o no es un DataFrame válido.")
    except NameError as e:
        print(f"Error: {e}")
else:
    print("No se cargó ningún diccionario...")


**Previsualizar registros depurados y eliminados (opcional):**

In [ ]:
# Previsualizar tabla de registros depurados (conservados) y verificar número de filas y columna
if cargar_diccionario.value != "No cargar diccionario":
  try:
    display(df_depurados_dicc)
    print(f"\nFilas/Columnas (shape) en registros conservados por diccionario: {df_depurados_dicc.shape}")
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

In [ ]:
# Previsualizar tabla de registros eliminados y verificar su número de filas y columnas
if cargar_diccionario.value != "No cargar diccionario":
  try:
    display(df_eliminados_dicc)
    print(f"\nFilas/Columnas (shape) en registros eliminados por diccionario: {df_eliminados_dicc.shape}")
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

In [ ]:
# Previsualizar data frame final
if cargar_diccionario.value != "No cargar diccionario":
    dfFinal = df_depurados_dicc
else:
    dfFinal = dfClean

display(dfFinal)

---

## **4. Análisis exploratorio de datos (visualizaciones de metadatos de videos o canales)**
*Se recomienda utilizar esta sección con conjuntos de datos no mayores a 20,000 filas.

### Gráfico de burbujas (por vistas y fecha de publicación/creación):

In [ ]:
# Graficar Bubble Chart por vistas y fecha de creación
df = dfFinal  # dataframe con los datos
df['viewCount'] = df['viewCount'].fillna(0)  # Fill NaN values with 0

# Verificamos si el dataframe tiene una columna 'videoId'
if 'videoId' in df.columns:
    # Si es una lista de videos, configuramos las codificaciones para el eje x e y
    x = 'publishedAt'
    y = 'viewCount'
    color = 'channelTitle'  # usaremos diferentes colores para cada título de canal
    hover_name = 'videoTitle'  # mostraremos el título del video en la información emergente
    title = f'YouTube: {df.shape[0]} videos por vistas y fecha de publicación'
else:
    # Si es una lista de canales, configuramos las codificaciones para el eje x e y
    x = 'publishedAt'
    y = 'viewCount'
    color = 'title'  # usaremos diferentes colores para cada título de canal
    hover_name = 'title'  # mostraremos el título del canal en la información emergente
    title = f'YouTube: {df.shape[0]} canales por vistas acumuladas y fecha de creación \n {project_name.value}'

# Convertimos la columna 'publishedAt' a objetos de fecha de Python
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

# Creamos el gráfico de burbujas con Plotly Express
fig = px.scatter(df, x=x, y=y, color=color, hover_name=hover_name, title=title,
                 size='viewCount',  # codificamos el tamaño de cada punto según la cantidad de vistas
                 opacity=0.6)  # configuramos la opacidad de cada punto al 60%


# Configuramos el eje x para que muestre fechas
fig.update_xaxes(title='Fecha de publicación', type='date')

# Configuramos el eje y para que muestre la cantidad de vistas
fig.update_yaxes(title='Cantidad de vistas')

fig.add_annotation(
    text=f'Datos: {project_name.value}',
    xref='paper',
    yref='paper',
    x=0,
    y=-0.2,
    showarrow=False,
    font_size=12,
    font_color='gray'
)

# Mostramos el gráfico
fig.show()

### Mapa de árbol (por canal y vistas):

*En el caso de listas de videos, la visualización muestra por default solo videos con más de 1000 vistas. Este umbral (`threshold`) puede ajustarse en el código y volverse a ejecutar.

In [ ]:
# Graficar mapa de árbol (treemap)
df = dfFinal  # dataframe con los datos
threshold = 1000 # para listas de videos, filtro por número de vistas

# Verificamos si el dataframe tiene una columna 'videoId'
if 'videoId' in df.columns:
    # Si es una lista de videos, configuramos las codificaciones para el treemap
    df_filtered = df[df['viewCount'] >= threshold]
    path = ['channelTitle', 'videoTitle']  # jerarquía para agrupar videos por canal
    values = 'viewCount'
    color = 'channelTitle'  # usaremos diferentes colores para cada título de canal
    title = f'YouTube: {df_filtered.shape[0]} videos por canal y título con más de {threshold} vistas'
else:
    # Si es una lista de canales, configuramos las codificaciones para el treemap
    df_filtered = df
    path = ['title']  # solo mostramos el título del canal
    values = 'viewCount'
    color = 'title'  # usaremos diferentes colores para cada título de canal
    title = f'YouTube: {df.shape[0]} canales por vistas acumuladas'

# Creamos el gráfico de treemap con Plotly Express
fig = px.treemap(df_filtered, path=path, values=values, color=color, title=title)

# Configuramos el título del gráfico
fig.update_layout(title=dict(y=0.95, x=0.5, xanchor='center'))

# Add-a-custom-annotation
fig.add_annotation(
    text=f'Datos: {project_name.value}',
    xref='paper',
    yref='paper',
    x=0,
    y=-0.2,
    showarrow=False,
    font_size=12,
    font_color='gray'
)

# Configurar despliegue de tamaño de texto
fig.update_traces(texttemplate='%{label}&lt;br&gt;%{value}', textinfo='label+value', textfont=dict(size=12))

# Mostramos el gráfico
fig.show()

### Gráfico de líneas múltiples (por vistas, canal y fecha de publicación de videos):
*Solo compatible con listas de videos

In [ ]:
# Gráfica de líneas múltiples. Alcance en vistas a videos por canal
df = dfFinal  # dataframe con los datos

# Verificamos si el dataframe tiene una columna 'videoId'
if 'videoId' in df.columns:
    # Si es una lista de videos, configuramos las codificaciones para el eje x e y
    x = 'publishedAt'
    y = 'viewCount'
    color = 'channelTitle'  # usaremos diferentes colores para cada ID de canal
    hover_name = 'videoTitle'  # mostraremos el título del video en la información emergente
    title = f'YouTube: {df.shape[0]} videos por vistas y fecha de publicación'

    # Ordenamos el dataframe por fecha de publicación y ID de canal
    df = df.sort_values(by=[x, 'channelId'])

else:
    # Si es una lista de canales, configuramos las codificaciones para el eje x e y
    x = 'publishedAt'
    y = 'viewCount'
    color = 'title'  # usaremos diferentes colores para cada ID de canal
    hover_name = 'title'  # mostraremos el título del canal en la información emergente
    title = f'YouTube: {df.shape[0]} canales por vistas acumuladas y fecha de creación \n {project_name.value}'

# Convertimos la columna 'publishedAt' a objetos de fecha de Python
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

# Creamos el gráfico de líneas múltiples con Plotly Express
fig = px.line(df, x=x, y=y, color=color, hover_name=hover_name, title=title)

# Configuramos la opacidad de las líneas
fig.update_traces(opacity=0.6)

# Configuramos el eje x para que muestre fechas
fig.update_xaxes(title='Fecha de publicación', type='date', fixedrange=True)

# Configuramos el eje y para que muestre la cantidad de vistas
fig.update_yaxes(title='Cantidad de vistas', fixedrange=True)

# Configuramos el color para que utilice un máximo de 64 colores
# fig.update_layout(coloraxis_colorbar_nticks=64)

fig.add_annotation(
    text=f'Datos: {project_name.value}',
    xref='paper',
    yref='paper',
    x=0,
    y=-0.2,
    showarrow=False,
    font_size=12,
    font_color='gray'
)

# Mostramos el gráfico
fig.show()

### Nube de palabras (por títulos de videos o canales, según el tipo de lista cargada):

In [ ]:
# Graficar Nube de Palabras
df = dfFinal  # dataframe con los datos

# Verificamos si el dataframe tiene una columna 'videoId'
if 'videoId' in df.columns:
    # Si es una lista de videos, utilizamos la columna 'videoTitle'
    text =''.join(df['sem_text'].tolist())
else:
    # Si es una lista de canales, utilizamos la columna 'channelTitle'
    text =''.join(df['sem_text'].tolist())

# Verificamos si el dataframe tiene una columna 'videoId'
wordcloudTitle = ''
if 'videoId' in df.columns:
  wordcloudTitle = f'YouTube: Palabras más repetidas en {df.shape[0]} nombres de videos'

else:
  wordcloudTitle = f'YouTube: Palabras más repetidas en {df.shape[0]} nombres de canales'



# Creamos el wordcloud
wordcloud = WordCloud(width=960, height=960, max_font_size=110).generate(text)

# Convertimos el wordcloud a una imagen
img = wordcloud.to_image()

# Creamos un gráfico interactivo con plotly
fig = go.Figure(data=[go.Image(z=img)])
fig.update_layout(title=wordcloudTitle, width=960, height=960)

# Mostramos el gráfico
fig.show()

### Bigrama (grafo de relaciones entre pares de palabras):

Definir función para identificar relaciones entre pares de palabras (bigramas) más frecuentes:

In [ ]:
# Definir función para calcular bigramas indicando un clúster específico u omitirlo para aplicarse con la muestra completa
def calculate_bigrams(df, nCluster=None):
    # Inicializar una lista para alojar todas las palabras en columna 'pregunta'
    all_words = []

    if nCluster is not None:
      dfW = df[df["cluster"] == nCluster]
      df = pd.DataFrame(dfW["sem_text"])
    else:
      df = pd.DataFrame(df["sem_text"])

    # Correr filtro por stopwords
    stop_words = set(stopwords.words('spanish'))

    for pregunta in df['sem_text']:
        # Tokenizar textos de columna pregunta text por palabra
        tokens = word_tokenize(pregunta, language='spanish')
        # Filtrar stopwords de tokens
        filtered_words = [word for word in tokens if word.lower() not in stop_words]
        all_words.extend(filtered_words)

    # Configurar identificador de bigramas
    bigram_measures = BigramAssocMeasures()
    bigram_finder = BigramCollocationFinder.from_words(all_words)

    # Calcular peso de bigramas utilizando frecuencia
    bigrams = bigram_finder.score_ngrams(bigram_measures.raw_freq)

    # Convertir bigramas calculados y su frecuencia en dataFrame
    bigrams_df = pd.DataFrame([(src, tgt, weight) for ((src, tgt), weight) in bigrams],
                              columns=['source', 'target', 'weight'])

    bigrams_df = bigrams_df.sort_values(by='weight', ascending=False)  # Ordenar bigramas por frecuencia

    return bigrams_df  # Regresar dataFrame con lista bigramas ordenados

Ejecutar función para identificar relaciones entre pares de palabras (bigramas) más frecuentes en toda la muestra:



In [ ]:
# Ejecutar cálculo de bigramas
bigrama_muestra = calculate_bigrams(df)

In [ ]:
bigrama_muestra

In [ ]:
top_bigrams_muestra = bigrama_muestra.head(20)

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 5))
plt.barh(top_bigrams_muestra['source'] + ' ' + top_bigrams_muestra['target'], top_bigrams_muestra['weight'], color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Bigrama')
plt.title(f'Top 20 Bigramas en Muestra Completa del Tema {project_name.value}')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
def visualize_bigrams(bigrams_df, top_n=100):
    # Crear un grafo dirigido
    G = nx.DiGraph()

    # Añadir nodos y bordes al grafo con pesos
    for _, row in bigrams_df.iterrows():
        G.add_edge(row['source'], row['target'], weight=row['weight'])

    # Calcular el grado ponderado de cada nodo
    weighted_degree = dict(G.degree(weight='weight'))

    # Ordenar nodos por grado ponderado y seleccionar los top_n
    top_nodes = sorted(weighted_degree.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_nodes = set(node for node, _ in top_nodes)

    # Crear un subgrafo con los nodos seleccionados
    H = G.subgraph(top_nodes).copy()

    # Obtener posiciones de los nodos usando el layout de Fruchterman-Reingold
    pos = nx.spring_layout(H, seed=42)

    # Obtener los bordes y los pesos para visualización
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )

    for edge in H.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += (x0, x1, None)
        edge_trace['y'] += (y0, y1, None)

    # Obtener los nodos para visualización
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers+text',
        textposition='top center',
        marker=dict(
            size=[],  # Dejar espacio para ajustar tamaño de nodos
            color='#1f78b4',
            line=dict(width=2, color='rgb(0,0,0)')
        )
    )

    for node in H.nodes():
        x, y = pos[node]
        node_trace['x'] += (x,)
        node_trace['y'] += (y,)
        node_trace['text'] += (node,)
        # Escalar el tamaño de los nodos por el grado ponderado
        node_trace['marker']['size'] += (weighted_degree[node] * 1000,)

    # Crear la visualización interactiva con Plotly
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=0)
                    ))

    fig.show()

In [ ]:
# Visualizar grafo con bigramas de top 100 nodos por grado con pesos
bigrams_df = calculate_bigrams(df)  # Calcular bigramas
visualize_bigrams(bigrams_df)  # Visualizar bigramas

---

## **5. Consolidar y exportar archivos de datos procesados (registros conservados y elminados por diccionario)**

### Exportar archivos CSV con tablas finales de registros depurados y eliminados:

In [ ]:
if cargar_diccionario.value != "No cargar diccionario":
  try:
    # Ejecutar concatenación de filas eliminadas por términos en diccionario
    df_eliminados_final = df_eliminados_dicc
    print(f"Filas/Columnas (shape) en registros totales eliminados: {df_eliminados_final.shape}")
    display(df_eliminados_final)


    # df_eliminados_final = pd.concat([df_eliminados_dicc, df_removed_duplicates], axis=0)
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

In [ ]:
# Previsualizar tabla de datos con registros conservados
dfFinal

**Exportar archivo de datos (en formato CSV) con registros conservados y procesados:**

In [ ]:
# Ejemplo exportar archivo de datos (CSV) con población de registros depurados
dfFinal.to_csv(f"{project_name.value}_registros-procesados.csv")

print(f"¡{project_name.value}_registros-procesados.csv descargado!")

**Exportar archivo de datos (en formato CSV) de registros eliminados por términos de descarte, con su razonamiento correspondiente (en caso de haber cargado algún diccionario):**

In [ ]:
# Ejemplo exportar archivo de datos (CSV) con registros eliminados por términos de descarte en diccionarios y repeticiones
if cargar_diccionario.value != "No cargar diccionario":
  try:
    df_eliminados_final.to_csv(f"{project_name.value}_registros-eliminados.csv")
    print(f"¡{project_name.value}_registros-eliminados.csv descargado!")
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

---

## **6. Referencias**
*  Bird, Steven, Edward Loper & Ewan Klein (2009).
Natural Language Processing with Python.  O'Reilly Media Inc.
* Kiss, T., & Strunk, J. (2006). Unsupervised Multilingual Sentence Boundary Detection. Computational Linguistics, 32(4), 485-525. https://doi.org/10.1162/coli.2006.32.4.485


*Programación asistida ocasionalmente con herramientas de IA Generativa: ChatGPT, Phind, Google Gemini y Perplexity

---

## **7. Créditos**
**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
Diego Arredondo Ortiz, José Luis Almendarez González y Javier de la Torre Silva

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Noviembre, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.
